<a href="https://colab.research.google.com/github/prabhatbajpai/Breaking-Bad/blob/main/January_Optimization_Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import glob as glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats

In [ ]:
df=pd.read_csv('./gdrive/My Drive/Deanonymization/WE_BC/WE_BC_January/WE_BC_Jan.csv')

In [ ]:
df=df.drop('hash',axis=1)
df=df.drop('time',axis=1)
df=df.drop('date',axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10209236 entries, 0 to 10209235
Data columns (total 24 columns):
 #   Column            Dtype  
---  ------            -----  
 0   balance           float64
 1   label             object 
 2   rec/sent          object 
 3   amount            float64
 4   block_id          int64  
 5   size              int64  
 6   weight            int64  
 7   version           int64  
 8   lock_time         int64  
 9   is_coinbase       int64  
 10  has_witness       int64  
 11  input_count       int64  
 12  output_count      int64  
 13  input_total       int64  
 14  input_total_usd   float64
 15  output_total      int64  
 16  output_total_usd  float64
 17  fee               int64  
 18  fee_usd           float64
 19  fee_per_kb        float64
 20  fee_per_kb_usd    float64
 21  fee_per_kwu       float64
 22  fee_per_kwu_usd   float64
 23  cdd_total         float64
dtypes: float64(10), int64(12), object(2)
memory usage: 1.8+ GB


In [ ]:
df=df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202036 entries, 65 to 10202009
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   balance           202036 non-null  float64
 1   label             202036 non-null  object 
 2   rec/sent          202036 non-null  object 
 3   amount            202036 non-null  float64
 4   block_id          202036 non-null  int64  
 5   size              202036 non-null  int64  
 6   weight            202036 non-null  int64  
 7   version           202036 non-null  int64  
 8   lock_time         202036 non-null  int64  
 9   is_coinbase       202036 non-null  int64  
 10  has_witness       202036 non-null  int64  
 11  input_count       202036 non-null  int64  
 12  output_count      202036 non-null  int64  
 13  input_total       202036 non-null  int64  
 14  input_total_usd   202036 non-null  float64
 15  output_total      202036 non-null  int64  
 16  output_total_usd 

In [ ]:
df['label'].value_counts()

Exchange        168925
Mining_pools     27838
Services          4663
Gambling           610
Name: label, dtype: int64

In [ ]:
from sklearn import preprocessing
def peenc(df):
  label_encoder=preprocessing.LabelEncoder()
  df['label']=label_encoder.fit_transform(df['label'])
  df['rec/sent']=label_encoder.fit_transform(df['rec/sent'])
peenc(df) 

In [ ]:
import warnings 
warnings.filterwarnings(action= 'ignore')

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef,classification_report,roc_curve
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
def f1():
  X=df.drop('label',axis=1)
  y=df['label']
  return X,y
X,y=f1()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
def f2():
  X,y=f1()
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
  norm = MinMaxScaler().fit(X_train)
  X_train = norm.transform(X_train)
  X_test = norm.transform(X_test)
  sm = SMOTE(random_state = 2)
  X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
  return X_train,X_test,y_train,y_test,X_train_res,y_train_res
X_train,X_test,y_train,y_test,X_train_res,y_train_res=f2()
def f3():
  return X_train,X_test,y_train,y_test,X_train_res,y_train_res;

In [ ]:
clf = RandomForestClassifier()
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=3,scoring='accuracy')
print("Accuracy:"+ str(scores.mean()))

Accuracy:0.8139235395051503


**1. Grid Search**

In [ ]:
#Random Forest
from sklearn.model_selection import GridSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': [10, 20, 30],
    #'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    #'min_samples_leaf': [1,2,4,8],
    #"bootstrap":[True,False],
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
grid = GridSearchCV(clf, rf_params, cv=3, scoring='accuracy')
grid.fit(X, y)
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))

{'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 20}
Accuracy:0.920395123502189


**2. Random Search**

In [ ]:
#Random Forest
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,64),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['gini','entropy']
}
n_iter_search=20 #number of iterations is set to 20, you can increase this number if time permits
clf = RandomForestClassifier(random_state=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='accuracy')
Random.fit(X, y)
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))

{'criterion': 'gini', 'max_depth': 49, 'max_features': 10, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 13}
Accuracy:0.9275473221877532


In [ ]:
!pip install git+https://github.com/scikit-optimize/scikit-optimize.git

  Cloning https://github.com/scikit-optimize/scikit-optimize.git to /tmp/pip-req-build-veexyuof
  Running command git clone -q https://github.com/scikit-optimize/scikit-optimize.git /tmp/pip-req-build-veexyuof
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/15/c4/1310a054d33abc318426a956e7d6df0df76a6ddfa9c66f6310274fb75d42/pyaml-20.4.0-py2.py3-none-any.whl
  Created wheel for scikit-optimize: filename=scikit_optimize-0.9.dev0-cp37-none-any.whl size=102086 sha256=5a3d17fee5b04612acefb9c9ce29475a32c48c874ddc7a7462d7c25d3852533d
  Stored in directory: /tmp/pip-ephem-wheel-cache-dix7vi3m/wheels/11/6f/86/2b772172db85ad0b4487d67e325e535ee8e7782b2a1dfcadf5
Successfully built scikit-optimize


In [ ]:
! pip install git+https://github.com/thuijskens/scikit-hyperband.git

  Cloning https://github.com/thuijskens/scikit-hyperband.git to /tmp/pip-req-build-842fhk_g
  Running command git clone -q https://github.com/thuijskens/scikit-hyperband.git /tmp/pip-req-build-842fhk_g
     |████████████████████████████████| 163kB 4.3MB/s 
  Created wheel for scikit-hyperband: filename=scikit_hyperband-0.0.1-cp37-none-any.whl size=10362 sha256=0a803646040c0ea0d654e45a03fb8d0186f43c83ff3038727232fc9d53764a97
  Stored in directory: /tmp/pip-ephem-wheel-cache-4_dgb5wk/wheels/9f/b0/a4/090ef56209908e0e7d583e47d3935f14593d31d9491b93cd3e
Successfully built scikit-hyperband


**3.Hyper Band**

In [ ]:
#Random Forest
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,23),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=10,max_iter=100,scoring='accuracy')
hyper.fit(X, y)
print(hyper.best_params_)
print("Accuracy:"+ str(hyper.best_score_))

{'criterion': 'entropy', 'max_depth': 11, 'max_features': 1, 'min_samples_leaf': 9, 'min_samples_split': 8, 'n_estimators': 33}
Accuracy:0.9374764893385338


**4. BO-GP**

Using BayseSearch_CV

In [ ]:
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': Integer(10,100),
    "max_features": Integer(1,23),
    'max_depth': Integer(5,50),
    "min_samples_split":Integer(2,11),
    "min_samples_leaf":Integer(1,11),
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, n_jobs=-1,scoring='accuracy')
#number of iterations is set to 20, you can increase this number if time permits
Bayes.fit(X, y)
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("Accuracy:"+ str(Bayes.best_score_))

OrderedDict([('criterion', 'gini'), ('max_depth', 5), ('max_features', 1), ('min_samples_leaf', 11), ('min_samples_split', 11), ('n_estimators', 10)])
Accuracy:0.930982597160902


Using skopt.gp_minimize

In [ ]:
#Random Forest
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = RandomForestClassifier()
# Define the hyperparameter configuration space
space  = [Integer(10, 100, name='n_estimators'),
            Integer(5, 50, name='max_depth'),
          Integer(1, 23, name='max_features'),
          Integer(2, 11, name='min_samples_split'),
          Integer(1, 11, name='min_samples_leaf'),
         Categorical(['gini', 'entropy'], name='criterion'),]
# Define the objective function
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="accuracy"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("Accuracy:%.4f" % -res_gp.fun)
print(res_gp.x)

Accuracy:0.9693
[100, 5, 1, 2, 5, 'entropy']


In [ ]:
!pip install  git+https://github.com/claesenm/optunity.git
#!echo "export PYTHONPATH=$PYTHONPATH:$(pwd)/optunity" >> ~/.bashrc


  Cloning https://github.com/claesenm/optunity.git to /tmp/pip-req-build-ii632npz
  Running command git clone -q https://github.com/claesenm/optunity.git /tmp/pip-req-build-ii632npz
  Created wheel for Optunity: filename=Optunity-1.1.1-cp37-none-any.whl size=74719 sha256=4b66b7e8ca7211fd762f06a71c843371b0adcd59b0f13fea79472b3a2758037c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3nzt8472/wheels/dc/76/e2/701e215c2b189821c60cb8dff4e5eafa62f0d85c7421090f04
Successfully built Optunity


In [ ]:
# pip install optunity

In [ ]:
# ! cd  usr/local/lib/python3.7/dist-packages/

**5.PSO**

In [ ]:
# #Random Forest
# import optunity
# import optunity.metrics

# data=X
# labels=y.tolist()
# # Define the hyperparameter configuration space
# search = {
#     'n_estimators': [10, 100],
#     'max_features': [1, 23],
#     'max_depth': [5,50],
#     "min_samples_split":[2,11],
#     "min_samples_leaf":[1,11],
#     "criterion":[0,1]
#          }
# # Define the objective function
# @optunity.cross_validated(x=data, y=labels, num_folds=3)
# def performance(x_train, y_train, x_test, y_test,n_estimators=None, max_features=None,max_depth=None,min_samples_split=None,min_samples_leaf=None,criterion=None):
#     # fit the model
#     if criterion<0.5:
#         cri='gini'
#     else:
#         cri='entropy'
#     model = RandomForestClassifier(n_estimators=int(n_estimators),
#                                    max_features=int(max_features),
#                                    max_depth=int(max_depth),
#                                    min_samples_split=int(min_samples_split),
#                                    min_samples_leaf=int(min_samples_leaf),
#                                    criterion=cri,
#                                   )
#     #predictions = model.predict(x_test)
#     scores=np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
#                                     scoring="accuracy"))
#     #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
#     return scores#optunity.metrics.accuracy(y_test, predictions)

# optimal_configuration, info, _ = optunity.maximize(performance,
#                                                   solver_name='particle swarm',
#                                                   num_evals=20,
#                                                    **search
#                                                   )
# print(optimal_configuration)
# print("Accuracy:"+ str(info.optimum))

**6. GA**

In [ ]:
! pip install sklearn-deap
# !pip install deap
# from deap import base
# from deap import creator
# from deap import tools

     |████████████████████████████████| 163kB 4.0MB/s 


In [ ]:
! pip install evolutionary-algorithm

     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for func-timeout: filename=func_timeout-4.3.5-cp37-none-any.whl size=15079 sha256=f5185d5c92855988312cb6ba02fa1c3818a83d2fdef54cb6cc05990a8cbf16b4
  Stored in directory: /root/.cache/pip/wheels/46/7c/4f/24f1d2d5bbff92219debe7ea19af84f76ddeb90dd4ec544f26
Successfully built func-timeout


In [ ]:
#Random Forest
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': np.logspace(1,1.8,num = 10 ,base=20,dtype='int'),
    'max_depth': np.logspace(1,2,num = 10 ,base=10,dtype='int'),
    "max_features":np.logspace(0.2,1,num = 5 ,base=8,dtype='int'),
    "min_samples_split":np.logspace(0.4, 1, num=5, base=10, dtype='int'), #[2, 3, 5, 7, 10],
    "min_samples_leaf":np.logspace(0.1,1,num = 5 ,base=11,dtype='int'),
    "criterion":['gini','entropy']
}
rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,23),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #Categorical(name='criterion', categories=['gini','entropy'])#
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
# Set the hyperparameters of GA 
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

Types [1, 1, 1, 1, 1, 1] and maxint [89, 21, 44, 8, 9, 1] detected
--- Evolve in 16038000 possible combinations ---
gen	nevals	avg     	min     	max     	std      
0  	10    	0.755357	0.724752	0.887461	0.0570316
1  	8     	0.790747	0.727271	0.893435	0.0771256
2  	5     	0.844217	0.727999	0.930715	0.0772707
3  	5     	0.858673	0.728024	0.893435	0.0652935
4  	7     	0.885955	0.824343	0.893435	0.0206077
5  	6     	0.893435	0.893435	0.893435	0        
Best individual is: {'n_estimators': 19, 'max_features': 2, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 6, 'criterion': 'entropy'}
with fitness: 0.9307153180621276
{'n_estimators': 19, 'max_features': 2, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 6, 'criterion': 'entropy'}
Accuracy:0.9307153180621276
